In [1]:
from data_loader import load_acc_data, load_params, load_vol_data
from classifiers import get_default_classifiers, set_classifier, get_online_predictions
from utilities import get_all_windows, extract_windows_data, apply_volume_filter, rename_columns, classify_by_access, evaluate_model

In [2]:
params = load_params()
df = load_acc_data("Pop2")
df_vol = load_vol_data("Pop2")
print(params)
df.head()

{'window_size': 1, 'step_size': 1, 'model': 'GP'}


,NameSpace,27_2013,28_2013,29_2013,30_2013,31_2013,32_2013,33_2013,34_2013,35_2013,...,12_2014,13_2014,14_2014,15_2014,16_2014,17_2014,18_2014,19_2014,20_2014,21_2014
0,0,2,21,152,20,25,0,1,0,29,...,0,1,9,6,0,0,3,7,13,10
1,1,15,1,0,0,0,0,0,2,6,...,0,0,0,0,0,0,0,0,0,0
2,2,955,2241,3801,2361,39,0,311,1432,3568,...,2356,2885,3322,3628,2001,2120,5981,4135,3621,1843
3,3,399,82,106,215,5,0,3,22,11,...,5,41,10,36,10,0,12,28,0,0
4,4,15,9,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
params['model'] = 'GP'

In [4]:
# exec.py

params = load_params()
params['model'] = 'GP'  # Defina o modelo como Programação Genética

if params['model'] != "ONL":
    clfs_dict = get_default_classifiers()
    clf = set_classifier(params['model'], clfs_dict)

    number_of_weeks = len(df.columns) - 1  # Ignorando a coluna de namespace
    last_week_of_training = number_of_weeks - params['window_size'] * 2 

    for first_week in range(0, last_week_of_training, params['step_size']):
        windows = get_all_windows(first_week, params['window_size'], params['step_size'])
        last_week_of_training = windows['x_train'][1]
        
        # Extrair os dados das janelas
        data_dict = extract_windows_data(df, df_vol, windows)
        
        # Aplicar o filtro de volume nos dados
        filtered_data = apply_volume_filter(data_dict, df_vol, last_week_of_training)
        
        # Renomear as colunas para serem consistentes
        renamed_data = rename_columns(filtered_data)
        
        # Transformar os dados de rótulo em binário
        renamed_data['y_train'] = classify_by_access(renamed_data['y_train'])
        renamed_data['y_test'] = classify_by_access(renamed_data['y_test'])
        
        # Treinar o modelo
        clf.fit(renamed_data['x_train'], renamed_data['y_train'])
        
        # Testar o modelo
        # Após obter as predições do modelo
        y_pred = clf.predict(renamed_data['x_test'])
        y_prob = clf.predict_proba(renamed_data['x_test'])[:, 1]  # Probabilidade da classe positiva

        # Adicionar as predições ao DataFrame
        renamed_data['y_test'] = renamed_data['y_test'].copy()
        renamed_data['y_test']['pred'] = y_pred

        # Adicionar colunas 'vol_bytes' e 'acc_fut' ao DataFrame de teste
        # Supondo que você tenha essas informações disponíveis
        # Se não tiver, precisa extrair dos dados originais
        renamed_data['y_test']['vol_bytes'] = df_vol.iloc[renamed_data['y_test'].index, :].sum(axis=1)
        renamed_data['y_test']['acc_fut'] = renamed_data['y_test'].sum(axis=1)

        # Calcular as métricas
        metrics = evaluate_model(renamed_data['y_test']['label'].values, y_pred, y_prob)

        # Calcular os custos
        costs = calculate_costs(renamed_data['y_test'])

        # Gerar resultados
        results = generate_results(renamed_data['y_test'], window=first_week, log_id='log_id', classifier_name=params['model'], FPPenaltyEnabled=True)

        # Imprimir ou salvar os resultados
        print(results)
else:
    # Código existente para o modo "ONL"
    pass


Geração 0: Melhor fitness = 0.8305084745762712
Geração 1: Melhor fitness = 0.8305084745762712
Geração 2: Melhor fitness = 0.8305084745762712
Geração 3: Melhor fitness = 0.8305084745762712
Geração 4: Melhor fitness = 0.8305084745762712
Geração 5: Melhor fitness = 0.8305084745762712
Geração 6: Melhor fitness = 0.8305084745762712
Geração 7: Melhor fitness = 0.8305084745762712
Geração 8: Melhor fitness = 0.8305084745762712
Geração 9: Melhor fitness = 0.8305084745762712
Geração 10: Melhor fitness = 0.8305084745762712
Geração 11: Melhor fitness = 0.8305084745762712
Geração 12: Melhor fitness = 0.8305084745762712
Geração 13: Melhor fitness = 0.8305084745762712
Geração 14: Melhor fitness = 0.8305084745762712
Geração 15: Melhor fitness = 0.8305084745762712
Geração 16: Melhor fitness = 0.8305084745762712
Geração 17: Melhor fitness = 0.8305084745762712
Geração 18: Melhor fitness = 0.8305084745762712
Geração 19: Melhor fitness = 0.8305084745762712
Geração 20: Melhor fitness = 0.8305084745762712
Ge

C:\Users\flavi\AppData\Local\Temp\ipykernel_6152\260820415.py:38: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  metrics = evaluate_model(renamed_data['y_test'].ravel(), y_pred, y_prob)


Geração 7: Melhor fitness = 0.6672694394213382
Geração 8: Melhor fitness = 0.6672694394213382
Geração 9: Melhor fitness = 0.6672694394213382
Geração 10: Melhor fitness = 0.6672694394213382
Geração 11: Melhor fitness = 0.6672694394213382
Geração 12: Melhor fitness = 0.6672694394213382
Geração 13: Melhor fitness = 0.6672694394213382
Geração 14: Melhor fitness = 0.6672694394213382
Geração 15: Melhor fitness = 0.6672694394213382
Geração 16: Melhor fitness = 0.6672694394213382
Geração 17: Melhor fitness = 0.6672694394213382
Geração 18: Melhor fitness = 0.6672694394213382
Geração 19: Melhor fitness = 0.6672694394213382
Geração 20: Melhor fitness = 0.6672694394213382
Geração 21: Melhor fitness = 0.6672694394213382
Geração 22: Melhor fitness = 0.6672694394213382
Geração 23: Melhor fitness = 0.6672694394213382
Geração 24: Melhor fitness = 0.6672694394213382
Geração 25: Melhor fitness = 0.6672694394213382
Geração 26: Melhor fitness = 0.6672694394213382
Geração 27: Melhor fitness = 0.667269439421

C:\Users\flavi\AppData\Local\Temp\ipykernel_6152\260820415.py:38: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  metrics = evaluate_model(renamed_data['y_test'].ravel(), y_pred, y_prob)


Geração 3: Melhor fitness = 0.6842857142857143
Geração 4: Melhor fitness = 0.6842857142857143
Geração 5: Melhor fitness = 0.6842857142857143
Geração 6: Melhor fitness = 0.6842857142857143
Geração 7: Melhor fitness = 0.6842857142857143
Geração 8: Melhor fitness = 0.6842857142857143
Geração 9: Melhor fitness = 0.6842857142857143
Geração 10: Melhor fitness = 0.6842857142857143
Geração 11: Melhor fitness = 0.6842857142857143
Geração 12: Melhor fitness = 0.6842857142857143
Geração 13: Melhor fitness = 0.6842857142857143
Geração 14: Melhor fitness = 0.6842857142857143
Geração 15: Melhor fitness = 0.6842857142857143
Geração 16: Melhor fitness = 0.6842857142857143
Geração 17: Melhor fitness = 0.6842857142857143
Geração 18: Melhor fitness = 0.6842857142857143
Geração 19: Melhor fitness = 0.6842857142857143
Geração 20: Melhor fitness = 0.6842857142857143
Geração 21: Melhor fitness = 0.6842857142857143
Geração 22: Melhor fitness = 0.6842857142857143
Geração 23: Melhor fitness = 0.6842857142857143

...

C:\Users\flavi\AppData\Local\Temp\ipykernel_6152\260820415.py:38: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  metrics = evaluate_model(renamed_data['y_test'].ravel(), y_pred, y_prob)


Geração 0: Melhor fitness = 0.8756076637117529
Geração 1: Melhor fitness = 0.8756076637117529
Geração 2: Melhor fitness = 0.8756076637117529
Geração 3: Melhor fitness = 0.8756076637117529
Geração 4: Melhor fitness = 0.8756076637117529
Geração 5: Melhor fitness = 0.8756076637117529
Geração 6: Melhor fitness = 0.8756076637117529
Geração 7: Melhor fitness = 0.8756076637117529
Geração 8: Melhor fitness = 0.8756076637117529
Geração 9: Melhor fitness = 0.8756076637117529
Geração 10: Melhor fitness = 0.8756076637117529
Geração 11: Melhor fitness = 0.8756076637117529
Geração 12: Melhor fitness = 0.8756076637117529
Geração 13: Melhor fitness = 0.8756076637117529
Geração 14: Melhor fitness = 0.8756076637117529
Geração 15: Melhor fitness = 0.8756076637117529
Geração 16: Melhor fitness = 0.8756076637117529
Geração 17: Melhor fitness = 0.8756076637117529
Geração 18: Melhor fitness = 0.8756076637117529
Geração 19: Melhor fitness = 0.8756076637117529
Geração 20: Melhor fitness = 0.8756076637117529
Ge

C:\Users\flavi\AppData\Local\Temp\ipykernel_6152\260820415.py:38: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  metrics = evaluate_model(renamed_data['y_test'].ravel(), y_pred, y_prob)


Geração 0: Melhor fitness = 0.8641488162344984
Geração 1: Melhor fitness = 0.8641488162344984
Geração 2: Melhor fitness = 0.8641488162344984
Geração 3: Melhor fitness = 0.8641488162344984
Geração 4: Melhor fitness = 0.8641488162344984
Geração 5: Melhor fitness = 0.8641488162344984
Geração 6: Melhor fitness = 0.8641488162344984
Geração 7: Melhor fitness = 0.8641488162344984


KeyboardInterrupt: 